In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import cv2,os
data_path='/content/drive/MyDrive/webapp/dataset'
categories=os.listdir(data_path)
labels=[i for i in range(len(categories))]

label_dict=dict(zip(categories,labels)) #empty dictionary
print(label_dict)
print(categories)
print(labels)

{'Jackfruit': 0, 'Mango': 1}
['Jackfruit', 'Mango']
[0, 1]


//Run only if you want to see the image   
//utilizes matplotlib

In [ ]:
import matplotlib.pyplot as plt


img_size=224

for category in categories:
    folder_path=os.path.join(data_path,category)
    img_names=os.listdir(folder_path)
        
    for img_name in img_names:
        img_path=os.path.join(folder_path,img_name)
        img=cv2.imread(img_path)
        resized=cv2.resize(img,(img_size,img_size))
        plt.imshow(img)
        break

//====

In [4]:
img_size=100
data=[]
target=[]

for category in categories:
    folder_path=os.path.join(data_path,category)
    img_names=os.listdir(folder_path)
        
    for img_name in img_names:
        img_path=os.path.join(folder_path,img_name)
        img=cv2.imread(img_path)

        try:  
            resized=cv2.resize(img,(img_size,img_size))
            #resizing the image  into 100x100, since we need a fixed common size for all the images in the dataset
            data.append(resized)
            target.append(label_dict[category])
            #appending the image and the label(categorized) into the list (dataset)
        except Exception as e:
            print('Exception:',e)
            #if any exception rasied, the exception will be printed here. And pass to the next image

In [ ]:
import random
random.shuffle(data)

In [ ]:
target[7]
data[7]

In [ ]:
import numpy as np
data=np.array(data)/255.0
data=np.reshape(data,(data.shape[0],img_size,img_size,3))
target=np.array(target)
from keras.utils import np_utils
new_target=np_utils.to_categorical(target)

In [ ]:
new_target.shape
data.shape
data.shape[1:]

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.layers import Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint

model=Sequential()

model.add(Conv2D(200,(3,3),input_shape=data.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#The first CNN layer followed by Relu and MaxPooling layers

model.add(Conv2D(100,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#The second convolution layer followed by Relu and MaxPooling layers

model.add(Flatten())
model.add(Dropout(0.5))
#Flatten layer to stack the output convolutions from second convolution layer
model.add(Dense(50,activation='relu'))
#Dense layer of 64 neurons
model.add(Dense(2,activation='softmax'))
#The Final layer with two outputs for two categories

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 98, 98, 200)       5600      
_________________________________________________________________
activation (Activation)      (None, 98, 98, 200)       0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 49, 49, 200)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 47, 47, 100)       180100    
_________________________________________________________________
activation_1 (Activation)    (None, 47, 47, 100)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 23, 23, 100)       0         
_________________________________________________________________
flatten (Flatten)            (None, 52900)             0

In [ ]:
from sklearn.model_selection import train_test_split
train_data,test_data,train_target,test_target=train_test_split(data,new_target,test_size=0.1)

In [ ]:
train_data.shape
train_target.shape
test_data.shape

In [ ]:
#checkpoint = ModelCheckpoint('model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')
history=model.fit(train_data,train_target,epochs=200,validation_split=0.2)

Epoch 1/200
13/13 [==============================] - 32s 2s/step - loss: 0.7806 - accuracy: 0.5429 - val_loss: 0.6647 - val_accuracy: 0.7071
Epoch 2/200
13/13 [==============================] - 32s 2s/step - loss: 0.6096 - accuracy: 0.6692 - val_loss: 0.5541 - val_accuracy: 0.7576
Epoch 3/200
13/13 [==============================] - 32s 2s/step - loss: 0.5339 - accuracy: 0.7551 - val_loss: 0.5583 - val_accuracy: 0.6970
Epoch 4/200
13/13 [==============================] - 32s 2s/step - loss: 0.5441 - accuracy: 0.7197 - val_loss: 0.5792 - val_accuracy: 0.7172
Epoch 5/200
13/13 [==============================] - 32s 2s/step - loss: 0.5053 - accuracy: 0.7753 - val_loss: 0.4568 - val_accuracy: 0.7677
Epoch 6/200
13/13 [==============================] - 32s 2s/step - loss: 0.4023 - accuracy: 0.8081 - val_loss: 0.4168 - val_accuracy: 0.7576
Epoch 7/200
13/13 [==============================] - 33s 3s/step - loss: 0.4407 - accuracy: 0.8081 - val_loss: 0.4052 - val_accuracy: 0.7879
Epoch 8/200
1

In [ ]:
model.save('model.h5')

In [1]:
from matplotlib import pyplot as plt
# plot the training loss and accuracy
N = 200
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), history.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), history.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), history.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), history.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="center right")
plt.savefig("CNN_Model")

NameError: ignored

<Figure size 432x288 with 0 Axes>